In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# YOLO 모델 로드
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# 이미지 로드
img = cv2.imread("example.jpg")
height, width, channels = img.shape

# 이미지 전처리 및 객체 감지
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# 객체 감지 후 클래스 분류
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # 객체의 경계 상자 좌표 계산
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # 이미지에서 객체 영역 추출
            roi = img[y:y+h, x:x+w]

            # 추출한 객체 영역을 CNN에 입력하여 클래스 분류
            model = load_model("classification_model.h5")
            # 이미지 전처리 등 필요한 전처리 수행
            roi_processed = preprocess(roi)
            # 클래스 분류
            predicted_class = model.predict(roi_processed)
            class_label = classes[np.argmax(predicted_class)]
            
            # 경계 상자 및 클래스 이름 그리기
            color = (255, 0, 0)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, class_label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# 결과 이미지 출력
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()